# Mini Lab: Using MCP to Access a Local File (Python + FastMCP)

In this lab, you will:

- Build a tiny **MCP server** in Python using FastMCP.
- Expose a **tool** that reads a local text file.
- Use an **MCP client** to call that tool and get the file contents.
- Understand how MCP standardizes access to tools and data.

This lab focuses on the *mechanics* of the Model Context Protocol (MCP), not on large language models.
You’ll see how an MCP server:
- Registers tools,
- Runs in the background,
- And responds to client requests.


In [ ]:
# Cell 2: Install FastMCP (MCP helper library) into this environment
# Run this once. Restart the kernel after installing.

%pip install fastmcp


In [ ]:
# Cell 3: Create a sample local text file for the MCP server to read
# We’ll create a simple text file in the current working directory so students don’t have to hunt for paths.

sample_filename = "sample_notes.txt"

sample_text = """\
This is a sample local file.

We are using it to demonstrate how an MCP server
can expose a tool that reads local files safely.

Students can edit this file and re-run the client
to see different results.
"""

with open(sample_filename, "w", encoding="utf-8") as f:
    f.write(sample_text)

print(f"✅ Created file: {sample_filename}\n")
print("File contents:\n")
print(sample_text)


In [ ]:
# Cell 4: Define a minimal MCP server with a tool to read a local file
# Here we use FastMCP to create a small MCP server in-process.
# It will have one tool: read_file(path: str) -> str.
# At this point, we have:
# - An MCP server object (mcp), with one tool read_file.
# - The tool uses normal Python file I/O.
# - FastMCP takes care of MCP protocol details (JSON-RPC, tool registration, etc.).

from fastmcp import FastMCP

# Create the MCP server instance
mcp = FastMCP("LocalFileServer")

@mcp.tool
def read_file(path: str) -> str:
    """
    Read a local text file and return its contents as a string.
    
    NOTE: For safety in a real system, we would add checks:
    - Restrict to a specific directory
    - Prevent reading very large files
    - Handle binary files carefully
    
    For this classroom lab, we assume simple, trusted text files.
    """
    try:
        with open(path, "r", encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError:
        return f"ERROR: File not found: {path}"
    except Exception as e:
        return f"ERROR reading file: {e}"

print("✅ MCP server defined with tool: read_file(path: str)")


In [ ]:
# Cell 5: Use a FastMCP client to call the server's read_file tool.
# We are not calling open() directly in this cell.
# Instead, we’re asking the MCP client to call a tool on a server (read_file) using the protocol.
# The server owns the tool; the client just requests it.

import asyncio
from fastmcp import Client

async def call_read_file(path: str):
    # Connect to the MCP server in-memory
    async with Client(mcp) as client:
        # Call the "read_file" tool with parameters
        result = await client.call_tool("read_file", {"path": path})
        return result

# Jupyter Notebook: directly await the coroutine
result = await call_read_file("sample_notes.txt")

print(f"📂 Requested file: sample_notes.txt\n")
print("📄 Tool result:\n")
print(result)

